In [1]:
# This notebook is for exploring how the API form NIH
# requirements: pip install xmltodict

In [2]:
import requests
import pandas as pd
import numpy as np
import xmltodict

A typical workflow would be to issue an ESearch call 
to get a list of unique identifiers, and then use the EFetch API to get full records.

In [3]:
#source: https://www.ncbi.nlm.nih.gov/books/NBK25498/#chapter3.Application_3_Retrieving_large

In [4]:
api_key = "857be3304fd504d7d4901fc8a7d12d221408" 

In [5]:
#assemble the esearch URL
query = 'mouse+brain+imaging'
base = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/'
url = base + "esearch.fcgi?db=pubmed&term=" + query + "&usehistory=y"

In [6]:
url

'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=mouse+brain+imaging&usehistory=y'

In [7]:
#execute the esearch URL to create a list of all search results 
#store it online behind the query key
output_esearch = requests.get(url)
dict_output_esearch = xmltodict.parse(output_esearch.content)

In [8]:
#parse WebEnv, QueryKey and Count (# records retrieved)
web = dict_output_esearch['eSearchResult']['WebEnv']
key = dict_output_esearch['eSearchResult']['QueryKey']
count = dict_output_esearch['eSearchResult']['Count']

In [9]:
web

'MCID_62f60ec1a34cba74ce2a9edb'

In [10]:
count

'21691'

In [11]:
retmax = 50
retstart = 0
count_test = 500

In [12]:
output_list = []

In [13]:
import timeit

In [14]:
start = timeit.timeit()

while retstart < count_test:
    efetch_url = base + "efetch.fcgi?db=pubmed&WebEnv="+web+"&query_key="+key+"&retstart="+str(retstart)+"&retmax="+str(retmax)+"&rettype=xml&retmode=xml"+"&api_key"+api_key
    output = requests.get(efetch_url)
    output_list.append(output)
    retstart += retmax

end = timeit.timeit()
print(end - start)

0.0035955009999995013


In [15]:
len(output_list)

10

In [34]:
output_list[19]

<Response [200]>

In [18]:
efetch_url = base + "efetch.fcgi?db=pubmed&WebEnv="+web+"&query_key="+key+"&retstart="+str(retstart)+"&retmax="+str(retmax)+"&rettype=xml&retmode=xml"+"&api_key"+api_key

In [19]:
efetch_url

'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&WebEnv=MCID_62f546cdb9bbb60eea077d28&query_key=1&retstart=0&retmax=10&rettype=xml&retmode=xml&api_key857be3304fd504d7d4901fc8a7d12d221408'

In [22]:
output = requests.get(efetch_url)

In [17]:
dict_output = xmltodict.parse(output_list[9].content)
dict_output

{'PubmedArticleSet': {'PubmedArticle': [{'MedlineCitation': {'@Status': 'PubMed-not-MEDLINE',
     '@Owner': 'NLM',
     'PMID': {'@Version': '1', '#text': '35474694'},
     'DateRevised': {'Year': '2022', 'Month': '07', 'Day': '16'},
     'Article': {'@PubModel': 'Electronic-eCollection',
      'Journal': {'ISSN': {'@IssnType': 'Electronic', '#text': '2667-2375'},
       'JournalIssue': {'@CitedMedium': 'Internet',
        'Volume': '1',
        'Issue': '1',
        'PubDate': {'Year': '2021', 'Month': 'May', 'Day': '24'}},
       'Title': 'Cell reports methods',
       'ISOAbbreviation': 'Cell Rep Methods'},
      'ArticleTitle': 'Comparative analysis of CI- and CIV-containing respiratory supercomplexes at single-cell resolution.',
      'Pagination': {'MedlinePgn': '100002'},
      'ELocationID': {'@EIdType': 'doi',
       '@ValidYN': 'Y',
       '#text': '10.1016/j.crmeth.2021.100002'},
      'Abstract': {'AbstractText': 'Mitochondria sustain the energy demand of the cell. The com

In [26]:
divided_dict = dict_output['PubmedArticleSet']['PubmedArticle']

In [93]:
meshs = divided_dict[4]['MedlineCitation']['MeshHeadingList']['MeshHeading']
keywords = ''
for mesh in range(len(meshs)):
    if mesh == 0:
        keywords+= meshs[mesh]['DescriptorName']['#text']
    else:
        keywords+= ", " + meshs[mesh]['DescriptorName']['#text']

In [94]:
keywords

'Aging, Animals, Cross-Sectional Studies, Hippocampus, Lasers, Longitudinal Studies, Mice, Microscopy, Confocal'

In [78]:
names

"Wang Marie Xun, Ismail Ozama, Braun Molly, Schindler Abigail G, Reemmer Jesica, Wang Zhongya, Haveliwala Mariya A, O'Boyle Ryan P, Han Warren Y, Roese Natalie, Grafe Marjorie, Woltjer Randall, Boison Detlev, Iliff Jeffrey J"

In [80]:
article_list = []
for i in range(len(divided_dict)):
    publication_date = divided_dict[i]['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']
    ## abstract:
    try:
        abstract = divided_dict[i]['MedlineCitation']['Article']['Abstract']['AbstractText']['#text']
    except:
        abstract = divided_dict[i]['MedlineCitation']['Article']['Abstract']['AbstractText']
    
    try:
        volume= divided_dict[i]['MedlineCitation']['Article']['Journal']['JournalIssue']['Volume']
    except:
        volume = '1'
        
    # authors list
    names = ''
    try:
        authors = divided_dict[i]['MedlineCitation']['Article']['AuthorList']['Author']
    except:
        authors = 'None'
        names = authors
    if authors!= "None":
        
        for j in range(len(authors)):
            try: 
                fullname = authors[j]['LastName'] + ' ' + authors[j]['ForeName']
                if j==0:
                    names = fullname
                else:
                    names+= ", " + fullname
            except:
                pass
    ## Mesh Words
    try:
        meshs = divided_dict[4]['MedlineCitation']['MeshHeadingList']['MeshHeading']
    except:
        meshs = "None"
    
    keywords = ''
    if meshs!="None":
        for mesh in range(len(meshs)):
            if mesh == 0:
                keywords+= meshs[mesh]['DescriptorName']['#text']
            else:
                keywords+= ", " + meshs[mesh]['DescriptorName']['#text']
    else:
        keywords=""
    article = {
        'PMID':divided_dict[i]['MedlineCitation']['PMID']['#text'],
        'abstract':abstract,
        'title':divided_dict[i]['MedlineCitation']['Article']['Journal']['Title'],
        'volume':volume,
        'pubDate':''.join([f'{value}-' for key, value in divided_dict[i]['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate'].items()])[:-1],
        'authors': names,
        'MeSh':keywords
    }
    article_list.append(article)

In [81]:
pd.DataFrame(article_list)

,PMID,abstract,title,volume,pubDate,authors
0,35474694,Mitochondria sustain the energy demand of the ...,Cell reports methods,1,2021-May-24,"Wischhof Lena, Scifo Enzo, Guranda Mihaela, Ja..."
1,35474242,Preclinical radiation research lacks standardi...,Scientific reports,12,2022-04-26,"Aitkenhead Adam H, Price Gareth J, Chadwick Am..."
2,35474185,"[{'@Label': 'BACKGROUND AND PURPOSE', '@NlmCat...",British journal of pharmacology,1,2022-Apr-26,"Chumak Tetyana, Quist Alexandra, Hermans Eva, ..."
3,35473943,"[{'@Label': 'BACKGROUND', '#text': 'Slowed cle...",Alzheimer's research & therapy,14,2022-04-26,"Wang Marie Xun, Ismail Ozama, Braun Molly, Sch..."
4,35473942,The hippocampus plays an important role in lea...,Scientific reports,12,2022-04-26,"Vázquez Alberdi Lucia, Martínez Mariana, Caler..."
5,35473251,Deep fluorescence imaging in mammalian brain t...,Optics express,30,2022-Apr-25,"Harms Fabrice, Hubert Antoine, Mercier Mathias..."
6,35472678,"[{'@Label': 'INTRODUCTION', 'sub': 'i', 'sup':...",Nuclear medicine and biology,110-111,2022 Jul-Aug,"Gu Jiwei, Jiang Hao, Liang Qianwa, Perlmutter ..."
7,35470553,"[{'@Label': 'BACKGROUND AND AIMS', '#text': 'S...",Addiction biology,27,2022-05,"Madularu Dan, Ortiz Richard J, Athanassiou Mar..."
8,35469845,Alzheimer's disease (AD) is a progressive neur...,Brain research,1788,2022-08-01,"Kim So Hee, Kim Yeong-Jin, Kim Hwan, Oh Youngs..."
9,35469103,"[{'@Label': 'BACKGROUND', '@NlmCategory': 'BAC...",Brain structure & function,227,2022-Jul,"Hawco Colin, Forde Natalie J, Jacobs Grace R, ..."


In [91]:
efetch_url = base + "efetch.fcgi?db=pubmed&WebEnv="+web+"&query_key="+key+"&retstart="+str(retstart)+"&retmax="+str(10)+"&rettype=xml&retmode=xml"+"&api_key"+api_key

In [105]:
efetch_url

'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&WebEnv=MCID_62f385fc2ad669188864eda0&query_key=1&retstart=0&retmax=10&rettype=xml&retmode=xml&api_key857be3304fd504d7d4901fc8a7d12d221408'

In [88]:
url2 = efetch_url
output2 = requests.get(url2)
dict_output2 = xmltodict.parse(output2.content)

In [13]:
dict_output_esearch

{'eSearchResult': {'Count': '21682',
  'RetMax': '20',
  'RetStart': '0',
  'QueryKey': '1',
  'WebEnv': 'MCID_62f5469d0f28845b326cfc80',
  'IdList': {'Id': ['35947077',
    '35945212',
    '35943780',
    '35943085',
    '35942567',
    '35941350',
    '35941286',
    '35939199',
    '35931210',
    '35931124',
    '35931075',
    '35930718',
    '35930672',
    '35928052',
    '35927993',
    '35927273',
    '35926451',
    '35926183',
    '35924840',
    '35922816']},
  'TranslationSet': {'Translation': [{'From': 'brain imaging',
     'To': '"neuroimaging"[MeSH Terms] OR "neuroimaging"[All Fields] OR ("brain"[All Fields] AND "imaging"[All Fields]) OR "brain imaging"[All Fields]'},
    {'From': 'mouse',
     'To': '"mice"[MeSH Terms] OR "mice"[All Fields] OR "mouse"[All Fields]'}]},
  'TranslationStack': {'TermSet': [{'Term': '"mice"[MeSH Terms]',
     'Field': 'MeSH Terms',
     'Count': '1736889',
     'Explode': 'Y'},
    {'Term': '"mice"[All Fields]',
     'Field': 'All Fields',


In [ ]:
#example efech
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=35943780